In [1]:
import os
from collections import Counter, defaultdict
import torch
from onconet.datasets.factory import RegisterDataset
from onconet.datasets.abstract_onco_dataset import Abstract_Onco_Dataset
import onconet.utils
import tqdm
from random import shuffle
import numpy as np
import datetime
import pdb

/hpc/packages/minerva-centos7/py_packages/3.7/lib/python3.7/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [4]:
SUMMARY_MSG = "Contructed CSV Mammo {} Survival {} dataset with {} records, {} exams, {} patients, and the following class balance \n {}"
MAX_VIEWS = 2
MAX_SIDES = 2
MAX_TIME=10

@RegisterDataset("csv_mammography")
class CSV_Mammography(Abstract_Onco_Dataset):

    '''
        Working dataset for suvival analysis. Note, does not support invasive cancer yet.
    '''
    def create_dataset(self, split_group, img_dir):
        """
        Return the dataset from the paths and labels in the json.

        :split_group: - ['train'|'dev'|'test'].
        :img_dir: - The path to the dir containing the images.
        """


        dict_dataset = defaultdict(dict)
        for _row in self.metadata_json:
            row = {k.encode('ascii', 'ignore').decode(): v.encode('ascii', 'ignore').decode() for k,v in _row.items()}
            patient_id, exam_id, split  = row['patient_id'], row['exam_id'], row['split_group']
            view = "{} {}".format(row['laterality'], row['view'])
            accession = "{}\t{}".format(patient_id, exam_id)
            file = row['file_path']

            dict_dataset[patient_id]['split'] = split
            dict_dataset[patient_id]['pid'] = patient_id
            if 'exams' not in dict_dataset[patient_id]:
                dict_dataset[patient_id]['exams'] = {}
            if accession not in dict_dataset[patient_id]['exams']:
                dict_dataset[patient_id]['exams'][accession] = {
                    'years_to_cancer': int(float(row['years_to_cancer'])),
                    'years_to_last_followup': int(float(row['years_to_last_followup'])),
                    'views': [],
                    'files': [],
                    'accession': accession
                }
            dict_dataset[patient_id]['exams'][accession]['views'].append(view)
            dict_dataset[patient_id]['exams'][accession]['files'].append(file)

        metadata = dict_dataset.values()
        dataset = []

        for mrn_row in tqdm.tqdm(metadata):
            ssn, exams = mrn_row['pid'],  mrn_row['exams']

            if mrn_row['split'] != split_group:
                continue

            for accession, exam in exams.items():

                if self.check_label(exam):
                    left_ccs, left_mlos, right_ccs, right_mlos = self.image_paths_by_views(exam)

                    if len(left_ccs+left_mlos+right_ccs+right_mlos) != 4:
                        continue

                    y, y_seq, y_mask, time_at_event = self.get_label(exam)

                    # Update with new data
                    all_images = ( right_ccs + right_mlos + left_ccs + left_mlos )
                    all_views =( [0]*len(right_ccs) + [1]*len(right_mlos) + [0]*len(left_ccs) + [1]*len(left_mlos) )
                    all_sides =  [0]*len(right_ccs) + [0]*len(right_mlos) + [1]*len(left_ccs) + [1]*len(left_mlos)
                    time_stamps =  [0]*len(all_images)

                    dataset.append({
                        'paths': pad_to_length(all_images, '<PAD>', self.args.num_images),
                        'y': y,
                        'y_mask': y_mask,
                        'y_seq': y_seq,
                        'time_at_event': time_at_event,
                        'exam': exam['accession'],
                        'ssn': ssn,
                        'time_seq': pad_to_length(time_stamps, MAX_TIME , self.args.num_images),
                        'view_seq': pad_to_length(all_views, MAX_VIEWS , self.args.num_images),
                        'side_seq': pad_to_length(all_sides, MAX_SIDES, self.args.num_images),
                        'additionals': [],
                        ### For back compatiblity with risk models that predict Left and right risk seperately
                        'year': -1,
                        'y_l': y,
                        'y_mask_l': y_mask,
                        'y_seq_l': y_seq,
                        'time_at_event_l': time_at_event,
                        'y_r': y,
                        'y_mask_r': y_mask,
                        'y_seq_r': y_seq,
                        'time_at_event_r': time_at_event
                    })
        return dataset

    def get_summary_statement(self, dataset, split_group):
        class_balance = Counter([d['y'] for d in dataset])
        exams = set([d['exam'] for d in dataset])
        patients = set([d['ssn'] for d in dataset])
        statement = SUMMARY_MSG.format(self.task, split_group, len(dataset), len(exams), len(patients), class_balance)
        statement += "\n" + "Censor Times: {}".format( Counter([d['time_at_event'] for d in dataset]))
        return statement

    def check_label(self, row):
        valid_pos = row['years_to_cancer'] < self.args.max_followup and row['years_to_cancer'] >= 0
        valid_neg = row['years_to_last_followup'] > 0

        return (valid_pos or valid_neg)

    def get_label(self, row):
        any_cancer = row["years_to_cancer"] < self.args.max_followup
        cancer_key = "years_to_cancer"

        y =  any_cancer
        y_seq = np.zeros(self.args.max_followup)

        if y:
            time_at_event = int(row[cancer_key])
            y_seq[time_at_event:] = 1

        else:
            time_at_event = int(min(row["years_to_last_followup"], self.args.max_followup) - 1)

        y_mask = np.array([1] * (time_at_event+1) + [0]* (self.args.max_followup - (time_at_event+1) ))
        #time_at_event = -2 --> (-2+1) + (5-(-2+1)) 
        assert len(y_mask) == self.args.max_followup
        return any_cancer, y_seq.astype('float64'), y_mask.astype('float64'), time_at_event

    @property
    def METADATA_FILENAME(self):
        return self.args.metadata_path

    @staticmethod
    def set_args(args):
        args.num_classes = 2
        args.max_followup = 5
        args.risk_factor_keys = ['density', 'binary_family_history', 'binary_biopsy_benign', 'binary_biopsy_LCIS', 'binary_biopsy_atypical_hyperplasia', 'age', 'menarche_age', 'menopause_age', 'first_pregnancy_age', 'prior_hist', 'race', 'parous', 'menopausal_status', 'weight','height', 'ovarian_cancer', 'ovarian_cancer_age', 'ashkenazi', 'brca', 'mom_bc_cancer_history', 'm_aunt_bc_cancer_history', 'p_aunt_bc_cancer_history', 'm_grandmother_bc_cancer_history', 'p_grantmother_bc_cancer_history', 'sister_bc_cancer_history', 'mom_oc_cancer_history', 'm_aunt_oc_cancer_history', 'p_aunt_oc_cancer_history', 'm_grandmother_oc_cancer_history', 'p_grantmother_oc_cancer_history', 'sister_oc_cancer_history', 'hrt_type', 'hrt_duration', 'hrt_years_ago_stopped']
        args.metadata_dir = None
        args.pred_risk_factors = True
        args.use_pred_risk_factors_at_test = True
        args.survival_analysis_setup = True
        args.num_images = 4
        args.multi_image = True
        args.min_num_images =  4
        args.class_bal = True
        args.test_image_transformers =  ["scale_2d", "align_to_left"]
        args.test_tensor_transformers =  ["force_num_chan_2d", "normalize_2d"]
        args.image_transformers =  ["scale_2d", "align_to_left", "rand_ver_flip", "rotate_range/min=-20/max=20"]
        args.tensor_transformers =  ["force_num_chan_2d", "normalize_2d"]

    @property
    def task(self):
        return "Risk"


def pad_to_length(arr, pad_token, max_length):
    arr = arr[-max_length:]
    return  np.array( [pad_token]* (max_length - len(arr)) + arr)


@RegisterDataset("csv_mammo_risk_full_future")
class CSV_Mammo_Cancer_Survival_Dataset(CSV_Mammography):

    '''
        Working dataset for suvival analysis. Note, does not support invasive cancer yet.
    '''
    def create_dataset(self, split_group, img_dir):
        """
        Return the dataset from the paths and labels in the json.

        :split_group: - ['train'|'dev'|'test'].
        :img_dir: - The path to the dir containing the images.
        """


        dict_dataset = defaultdict(dict)
        for _row in self.metadata_json:
            row = {k.encode('ascii', 'ignore').decode(): v.encode('ascii', 'ignore').decode() for k,v in _row.items()}
            patient_id, exam_id, split  = row['patient_id'], row['exam_id'], row['split_group']
            view = "{} {}".format(row['laterality'], row['view'])
            accession = "{}\t{}".format(patient_id, exam_id)
            file = row['file_path']

            dict_dataset[patient_id]['split'] = split
            dict_dataset[patient_id]['pid'] = patient_id
            if 'exams' not in dict_dataset[patient_id]:
                dict_dataset[patient_id]['exams'] = {}
            if accession not in dict_dataset[patient_id]['exams']:
                dict_dataset[patient_id]['exams'][accession] = {
                    'years_to_cancer': int(float(row['years_to_cancer'])),
                    'years_to_last_followup': int(float(row['years_to_last_followup'])),
                    'views': [],
                    'files': [],
                    'accession': accession
                }
            dict_dataset[patient_id]['exams'][accession]['views'].append(view)
            dict_dataset[patient_id]['exams'][accession]['files'].append(file)

        metadata = dict_dataset.values()
        dataset = []

        for mrn_row in tqdm.tqdm(metadata):
            ssn, exams = mrn_row['pid'],  mrn_row['exams']

            if mrn_row['split'] != split_group:
                continue

            for accession, exam in exams.items():

                if self.check_label(exam):
                    left_ccs, left_mlos, right_ccs, right_mlos = self.image_paths_by_views(exam)



                    y, y_seq, y_mask, time_at_event = self.get_label(exam)

                    for path in  left_ccs + left_mlos + right_ccs + right_mlos:
                        dataset.append({
                            'path': path,
                            'y': y,
                            'y_mask': y_mask,
                            'y_seq': y_seq,
                            'time_at_event': time_at_event,
                            'exam': exam['accession'],
                            'ssn': ssn,
                            'additional': {},
                            ### For back compatiblity with risk models that predict Left and right risk seperately
                            'year': -1,
                            'y_l': y,
                            'y_mask_l': y_mask,
                            'y_seq_l': y_seq,
                            'time_at_event_l': time_at_event,
                            'y_r': y,
                            'y_mask_r': y_mask,
                            'y_seq_r': y_seq,
                            'time_at_event_r': time_at_event
                        })
        return dataset

    @staticmethod
    def set_args(args):
        args.num_classes = 2
        args.max_followup = 5
        args.risk_factor_keys = ['density', 'binary_family_history', 'binary_biopsy_benign', 'binary_biopsy_LCIS', 'binary_biopsy_atypical_hyperplasia', 'age', 'menarche_age', 'menopause_age', 'first_pregnancy_age', 'prior_hist', 'race', 'parous', 'menopausal_status', 'weight','height', 'ovarian_cancer', 'ovarian_cancer_age', 'ashkenazi', 'brca', 'mom_bc_cancer_history', 'm_aunt_bc_cancer_history', 'p_aunt_bc_cancer_history', 'm_grandmother_bc_cancer_history', 'p_grantmother_bc_cancer_history', 'sister_bc_cancer_history', 'mom_oc_cancer_history', 'm_aunt_oc_cancer_history', 'p_aunt_oc_cancer_history', 'm_grandmother_oc_cancer_history', 'p_grantmother_oc_cancer_history', 'sister_oc_cancer_history', 'hrt_type', 'hrt_duration', 'hrt_years_ago_stopped']
        args.metadata_dir = None
        args.pred_risk_factors = True
        args.use_pred_risk_factors_at_test = True
        args.survival_analysis_setup = True
        args.class_bal = True
        args.test_image_transformers =  ["scale_2d", "align_to_left"]
        args.test_tensor_transformers =  ["force_num_chan_2d", "normalize_2d"]
        args.image_transformers =  ["scale_2d", "align_to_left", "rand_ver_flip", "rotate_range/min=-20/max=20"]
        args.tensor_transformers =  ["force_num_chan_2d", "normalize_2d"]




In [2]:
metadata = "/sc/arion/projects/shenl03_ml/2023_adriana_bc_risk/Mirai_test/subset_mssm_mammogram_metadata_for_mirai.csv"

In [3]:
import pandas as pd
metadata_file = pd.read_csv(metadata)
metadata_file.head()
# Subset for particular values in a column
#subset_df = metadata_file[metadata_file['exam_id'] <= 4]

In [5]:
max(metadata_file["years_to_last_followup"]), min(metadata_file["years_to_last_followup"])
metadata_file.shape
metadata_file.notnull().all()

0

In [8]:
max_followup = 5

In [14]:
def get_label(self, row):
    any_cancer = row["years_to_cancer"] < max_followup
    cancer_key = "years_to_cancer"
    
    y =  any_cancer
    y_seq = np.zeros(max_followup)
    
    if y:
        time_at_event = int(row[cancer_key])
        y_seq[time_at_event:] = 1

    else:
        time_at_event = int(min(row["years_to_last_followup"], max_followup) - 1)
        #print(time_at_event)
    
    print(time_at_event)
    #y_mask = np.array([1] * (time_at_event+1) + [0]* (max_followup - (time_at_event+1) ))
    #assert len(y_mask) == max_followup
    #return any_cancer, y_seq.astype('float64'), y_mask.astype('float64'), time_at_event

In [34]:
max(metadata_file["years_to_cancer"]), min(metadata_file["years_to_cancer"])
subset_minus2 = metadata_file[metadata_file['years_to_cancer'] <0]
subset_minus2

subset_ptn_634585 = metadata_file[metadata_file['patient_id'] == 634585]
subset_ptn_634585
## look at only exam_id = 1
## look at this patient 634585

,patient_id,exam_id,laterality,view,file_path,years_to_cancer,years_to_last_followup,split_group
2430,634585,1,L,CC,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,2,0,test
2431,634585,1,L,CC,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,2,0,test
2432,634585,1,L,MLO,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,2,0,test
2433,634585,1,L,MLO,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,2,0,test
2434,634585,1,L,MLO,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,2,0,test
2435,634585,1,R,CC,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,2,0,test
2436,634585,1,R,CC,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,2,0,test
2437,634585,1,R,MLO,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,2,0,test
2438,634585,1,R,MLO,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,2,0,test
2439,634585,1,R,MLO,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,2,0,test


In [5]:
##checking data file for 634585
data = "/sc/arion/projects/shenl03_ml/2023_adriana_bc_risk/Mirai_test/all_mssm_mammogram_metadata_for_mirai.csv"
import pandas as pd
data_file = pd.read_csv(data)
data_file
pt_634585 = data_file[data_file['subjectId'] == 634585]
pt_634585

#The issue with this patient is that examIndex occurs after daysSincePreviousExam (not aligned)

,subjectId,examIndex,daysSincePreviousExam,cancerL,cancerR,invL,invR,age,implantEver,implantNow,...,assessR_DREAM_r2,imageIndex,view,laterality,filename,file_path,years_to_last_followup,year_of_mammogram,combined_L_R,years_to_cancer
2437,634585,1,0,0,0,0,0,48,0,.,...,1/2,1,CC,L,00007666.dcm,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,0.000000,2008,0,2
2438,634585,1,0,0,0,0,0,48,0,.,...,1/2,2,CC,L,00007667.dcm,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,0.000000,2008,0,2
2439,634585,1,0,0,0,0,0,48,0,.,...,1/2,3,MLO,L,00007671.dcm,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,0.000000,2008,0,2
2440,634585,1,0,0,0,0,0,48,0,.,...,1/2,4,MLO,L,00007672.dcm,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,0.000000,2008,0,2
2441,634585,1,0,0,0,0,0,48,0,.,...,1/2,5,MLO,L,00007673.dcm,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,0.000000,2008,0,2
2442,634585,1,0,0,0,0,0,48,0,.,...,1/2,6,CC,R,00007664.dcm,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,0.000000,2008,0,2
2443,634585,1,0,0,0,0,0,48,0,.,...,1/2,7,CC,R,00007665.dcm,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,0.000000,2008,0,2
2444,634585,1,0,0,0,0,0,48,0,.,...,1/2,8,MLO,R,00007668.dcm,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,0.000000,2008,0,2
2445,634585,1,0,0,0,0,0,48,0,.,...,1/2,9,MLO,R,00007669.dcm,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,0.000000,2008,0,2
2446,634585,1,0,0,0,0,0,48,0,.,...,1/2,10,MLO,R,00007670.dcm,/sc/arion/projects/Nestlerlab/shenl03_ml/2023_...,0.000000,2008,0,2


In [29]:
def get_label(self, row):
    max_followup = 5
    any_cancer = row["years_to_cancer"] < max_followup
    cancer_key = "years_to_cancer"
    y =  any_cancer
    y_seq = np.zeros(max_followup)
    if y:
        time_at_event = int(row[cancer_key])
        y_seq[time_at_event:] = 1
        if time_at_event < 0:
            print("time_less_0")
    else:
        time_at_event = int(min(row["years_to_last_followup"], max_followup) - 1)
        
    #if time_at_event >= 5:
        #print("time_greater_5")
        
    #if time_at_event < 0:
        #print("time_less_0")
        
    y_mask = np.array([1] * (time_at_event+1) + [0]* (max_followup - (time_at_event+1) ))
    #if len(y_mask) != 5:
        #print("no")
        
    y_mask_list.append(y_mask)
        
    return y_mask_list
    
    #print(y_mask, max_followup)
    #assert len(y_mask) == max_followup

In [30]:
metadata_file["years_to_cancer"].dtype

dtype('int64')

In [99]:
max(subset_df["exam_id"])

4

In [57]:
max(metadata_file["years_to_last_followup"])

5

In [30]:
y_mask_list = []

for index,row in metadata_file.iterrows():
    get_label(metadata_file, row)

time_less_0
time_less_0
time_less_0
time_less_0
time_less_0
time_less_0
time_less_0
time_less_0
time_less_0


In [203]:
len(y_mask_list)

13933

In [13]:
y_mask_list

for i, arr in enumerate(y_mask_list):
    if len(arr) > 5:
        print("Array at iteration", i, "has more than 5 elements:", arr)

In [194]:
check_this = subset_df[subset_df["patient_id"] == 831895]
check_this

subset_df.loc[2142]

check_this = subset_df[subset_df["patient_id"] == 639826]
check_this

subset_df_df.loc[2138]

patient_id                                                           545006
exam_id                                                                   1
laterality                                                                L
view                                                                     CC
file_path                 /sc/arion/projects/Nestlerlab/shenl03_ml/2023_...
years_to_cancer                                                         100
years_to_last_followup                                                    0
split_group                                                            test
Name: 2138, dtype: object

In [1]:
import sklearn
print (sklearn.__version__)

1.0.1
